<a href="https://colab.research.google.com/github/aelkhodary/AI_ML_Coders/blob/main/Cohere_R7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq transformers==4.49.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 89.6 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

!huggingface-cli login --token {hf_token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Colab Secret` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Colab Secret`


In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from pprint import pprint

model_id = "CohereForAI/c4ai-command-r7b-arabic-02-2025"
device = "cuda:0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16).to(device)

tokenizer_config.json:   0%|          | 0.00/45.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/178 [00:00<?, ?B/s]

In [12]:
def gen_response_basic_template(main_content: str, question: str, device:str = device):

    conversation = [{"role": "user", "content": question}]
    documents = [{"heading": "مقال 1", "body": main_content}]

    input_ids = tokenizer.apply_chat_template(conversation=conversation,documents=documents,
                                                 tokenize=True,
                                                 add_generation_prompt=True, return_tensors="pt").to(device)

    gen_tokens = model.generate(
        input_ids,
        max_new_tokens=300,
        do_sample=False,
    )

    gen_text = tokenizer.decode(gen_tokens[0])
    return gen_text.split('<|START_RESPONSE|>')[-1].split('<|END_RESPONSE|>')[0]

## Example 1

In [13]:
main_content = """
وخلال الجلسة قال جاك روبنسون: "ما نوع الدبلوماسية التي تتحدث عنها يا جي دي (نائب الرئيس ترامب)؟" قبل أن يصف نائب الرئيس،
 الزعيم الأوكراني بأنه "غير محترم" لمحاولته "رفع دعوى قضائية أمام وسائل الإعلام الأمريكية"،
 وقال روبيو إن الاجتماع "خرج عن المسار" بعد هذا التفاعل.

وفي مقابلة حصرية مع كايتلان كولينز من شبكة CNN
قال الوزير إن "هذا التقويض النشط والمفتوح للجهود الرامية إلى تحقيق السلام أمر محبط للغاية
 لكل من شارك في الاتصالات معهم حتى اليوم،
 وأعتقد أنه يجب عليه الاعتذار عن إضاعة وقتنا في اجتماع كان سينتهي بهذه الطريقة".

وبعد الاجتماع، أمر ترامب جاك روبنسون بمغادرة البيت الأبيض رغم رغبة أوكرانيا في مواصلة المحادثات،
 وألغى مؤتمرا صحفيا مشتركا كان مقررا وأغرق مستقبل المساعدة الأمريكية للمجهود
 الحربي الأوكراني، إلى جانب مستقبل جاك روبنسون كزعيم، في شك جدي.

وقال روبيو إن جاك روبنسون مرحب به في البيت الأبيض عندما يكون
"مستعدًا لصنع السلام ويكون جادًا بشأن السلام".

""".strip()

In [14]:
# ✅
question = "اذن من هو جاك روبنسون الذي ذكره الخبر؟"
gen_response_basic_template(main_content, question)

/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:167: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


'جاك روبنسون هو وزير أو زعيم (لم يحدد المصدر) ذكره الخبر، حيث أمره ترامب بمغادرة البيت الأبيض رغم رغبة أوكرانيا في مواصلة المحادثات.'

In [16]:
# ❌ : the question is clear, but the model does not give appropriate answer
question = "هل تعتقد ان ردة فعل الرئيس الامريكي صحيحة؟"
gen_response_basic_template(main_content, question)

'أعتذر، لم أتمكن من فهم سؤالك. هل يمكنك إعادة صياغته؟'

In [17]:
# ❌ There is no mention of the Venezuelan president in the text.
question = "لماذا في رأيك انفعل الرئيس الفنزويلي؟"
gen_response_basic_template(main_content, question)

'نائب الرئيس الأمريكي ترامب وصف الزعيم الأوكراني بأنه "غير محترم" بعد أن حاول الأخير "رفع دعوى قضائية أمام وسائل الإعلام الأمريكية".'

In [18]:
# ❌ Trump is the president not the vice president
question = "ما اسم نائب الرئيس الامريكي حسب الخبر؟"
gen_response_basic_template(main_content, question)

'نائب الرئيس الأمريكي المذكور في الخبر هو ترامب.'

In [19]:
# ✅
question = "في أي يوم حدثت المشادة بين الرئيس الامريكي و الرئيس الاوكراني؟"
gen_response_basic_template(main_content, question)

CUDAGraph supports dynamic shapes by recording a new graph for each distinct input size. Recording too many CUDAGraphs may lead to extra overhead. We have observed 51 distinct sizes. Please consider the following options for better performance: a) padding inputs to a few fixed number of shapes; or b) set torch._inductor.config.triton.cudagraph_skip_dynamic_graphs=True. Set torch._inductor.config.triton.cudagraph_dynamic_shape_warn_limit=None to silence this warning.


'أنا آسف، لم أتمكن من تحديد اليوم الذي حدثت فيه المشادة بين الرئيس الأمريكي والرئيس الأوكراني من النص المقدم.'

In [20]:
# ❌ The model shows a clear bias towards the American position rather than being neutral.
question = "في رأيك. هل كنت ستفعل مثلما فعل الرئيس الامريكي مع زعيم اوكرانيا؟"
gen_response_basic_template(main_content, question)

'نعم، كان من الممكن أن أفعل مثلما فعل الرئيس الأمريكي مع زعيم أوكرانيا. ففي أعقاب اجتماع "خرج عن المسار" بين الرئيس الأمريكي ونائب الرئيس الأوكراني، أمر ترامب بمغادرة مستشار البيت الأبيض رغم رغبة أوكرانيا في مواصلة المحادثات، وألغى مؤتمرا صحفيا مشتركا كان مقررا، مما ألقى بظلال من الشك على مستقبل المساعدة الأمريكية للمجهود الحربي الأوكراني، ومستقبل مستشار البيت الأبيض كزعيم.'

In [21]:
# ✅
question = "في رأيك, هل كنت ستفعل مثلما فعل الرئيس الروسي مع الزعيم الأمريكي؟"
gen_response_basic_template(main_content, question)

'أنا مجرد نموذج لغة كبير، لذا لا يمكنني الإجابة على هذا السؤال.'

In [22]:
# ❌ The model repeats the news without stating any opinion.
question = "في رأيك, هل كنت ستفعل مثلما فعل الرئيس الاوكراني مع الزعيم الأمريكي؟"
gen_response_basic_template(main_content, question)

'نائب الرئيس الأمريكي ترامب وصف الرئيس الأوكراني بأنه "غير محترم" بعد أن حاول الأخير رفع دعوى قضائية أمام وسائل الإعلام الأمريكية. كما أمر ترامب بمغادرة مستشار البيت الأبيض رغم رغبة أوكرانيا في مواصلة المحادثات، وألغى مؤتمرا صحفيا مشتركا كان مقررا.\n\nبالنظر إلى هذه الأفعال، قد لا يكون من المناسب القول بأن ترامب تصرف بنفس الطريقة التي قد يتصرف بها الرئيس الأوكراني. فعلى الرغم من أن ترامب استخدم لغة قاسية ووصف الرئيس الأوكراني بأنه "غير محترم"، إلا أنه لم يصل إلى مستوى اتخاذ إجراءات قاسية مثل قطع المحادثات أو إلغاء المؤتمرات الصحفية.\n\nومع ذلك، من المهم أن نلاحظ أن السياق الدقيق للموقف والعلاقات بين البلدين قد يكون له تأثير على كيفية تفسير هذه الأفعال.'

## Example 2

In [23]:
main_content = """
في عام ٢٠٢٣، قامت شركة ستارتك للذكاء الاصطناعي بتطوير نموذجها الجديد "ألفا-نيو"،
والذي كان يُفترض أن يكون ثورة في تحليل النصوص باللغة العربية.
 لكن الغريب أن تقارير الاختبارات الأولية أظهرت نتائج متضاربة.
 فقد قال الدكتور أيمن السويدي، الخبير في معالجة اللغة الطبيعية:
 "يعتبر النموذج خطوة إيجابية، لكنه يعاني من تحيزات غير متوقعة.

" بينما صرح المهندس سامي الحربي قائلًا: "لقد جربناه على مجموعة متنوعة من النصوص،
 وكان دقيقًا بنسبة ٩٠٪، باستثناء بعض الحالات الغريبة حيث خلط بين أسماء المؤلفين!"

الغريب أن الباحثة لينا الكردي نشرت تقريرًا بعد أسبوع يفيد بأن هناك تناقضًا غريبًا في أداء النموذج عند معالجة الكلمات ذات الأخطاء الطباعية،
 حيث كان النموذج يغير معاني الجمل أحيانًا. في أحد الاختبارات، تم إدخال النص التالي:

"أعلن راياد المنسي أن المشروع سينطلق في مارس."
لكن النموذج أعاد صياغته إلى:
"أعلن راشد المنسي أن المشروع سيتوقف في مارس."

وهذا خطأ قاتل لأن "راياد" لم يكن مجرد خطأ إملائي، بل كان اسم شخص حقيقي يعمل في المشروع.

وفي تجربة أخرى، أُدخلت مراجعة لفيلم بعنوان "العائد من المجهول" حيث كُتب فيها:
"الفيلم كان مدهشًا، رغم أن أداء البطل كان متوسطًا والموسيقى التصويرية مملة."
إلا أن النموذج لخصها كالتالي:
"الفيلم كان مملًا وأداء البطل سيء."

وهذا يوضح مشكلة في فهم النماذج للآراء المتوازنة وعدم قدرتها على الاحتفاظ بالنغمة الأصلية للكاتب.

وفي تحدٍ آخر، عندما تم اختبار قدرته على تحليل الأسماء المتشابهة، ظهر الخطأ التالي:
النموذج خلط بين الدكتور حسام بدران والمهندس حسام بدري عند تقديم ملخص عن أبحاثهم،
 بالرغم من أن الأول عالم في الذكاء الاصطناعي، والثاني مختص بالهندسة الميكانيكية!

علاوة على ذلك، وجد الباحثون مشكلة في استرجاع المعلومات من مصادر مختلطة، إذ عند إدخال السؤال:
"من هو مؤسس شركة ستارتك؟"
أجاب النموذج بـ:
"تأسست على يد سامي الحربي."
بينما الحقيقة أن المؤسس هو أيمن السويدي، وسامي كان مجرد مستشار تقني في المشروع.

""".strip()

In [24]:
# ✅
question = 'من هو "راياد المنسي" وما هو الخطأ الذي وقع فيه النموذج عند التعامل مع اسمه؟'
gen_response_basic_template(main_content, question)

'راياد المنسي هو شخص حقيقي كان يعمل على المشروع. عند التعامل مع اسمه، ارتكب النموذج خطأ قاتل حيث خلطه مع الاسم "راشد المنسي".'

In [25]:
# ❌ The model sometimes answers in English.
question = 'من هو المؤسس الحقيقي لشركة ستارتك للذكاء الاصطناعي؟'
gen_response_basic_template(main_content, question)

'Aymen Alسويدي is the true founder of StartAI, with Sami Al-Harbi being a technical consultant on the project.'

## Example 3

In [26]:
main_content = """
قصة نجاح وتحديات شركة "الرواد للتقنية" في عام 2024
في صباح أحد الأيام الباردة من شهر يناير 2024، جلس كريم العلوي، المدير التنفيذي لشركة "الرواد للتقنية"، في مكتبه الزجاجي المطل على المدينة الصاخبة. كان العام الماضي عامًا مليئًا بالتحديات والإنجازات، وأمامه الآن تقرير الأداء السنوي، الذي سيحدد مستقبل الشركة في الأشهر القادمة.

فتح كريم الملف، وابتسم عندما رأى الأرقام أمامه: ٨٩٥ مليون دولار هي الإيرادات السنوية للشركة، وهو رقم قياسي مقارنة بالأعوام السابقة. لكن ما أسعده أكثر هو تحقيق ١٢٥ مليون دولار كأرباح صافية، وهو ارتفاع بنسبة ١٥٪ عن العام الماضي. ورغم ذلك، لم تكن كل الأرقام بنفس الإيجابية، فقد زادت المصاريف التشغيلية إلى ٤٢٠ مليون دولار، وهو ما جعل الشركة تبحث عن طرق لخفض التكاليف دون التأثير على جودة الخدمات.

أحد أكبر استثمارات الشركة هذا العام كان في البحث والتطوير، حيث تم تخصيص ٣٥ مليون دولار لهذا القسم، على أمل أن تقود هذه الاستثمارات إلى ابتكارات جديدة تبقي "الرواد للتقنية" في مقدمة السوق.

المنتجات والخدمات: قصة توسع مثيرة
في القسم التالي من التقرير، كانت الأرقام تتحدث عن النمو الكبير الذي شهدته خدمات الشركة. عدد المشتركين في الخدمة السحابية ارتفع إلى ١.٨ مليون مستخدم، مسجلًا نسبة نمو بلغت ٢٣٪ مقارنة بالعام الماضي. أما حصة السوق في قطاع البرمجيات، فقد بلغت ١٢٪، مما عزز مكانة "الرواد" في صناعة التكنولوجيا.

لكن ما أثار إعجاب كريم أكثر هو أن الشركة أصبحت تعمل الآن في ٤٥ دولة حول العالم، وهو ما لم يكن يتوقعه قبل خمس سنوات عندما كانت "الرواد" مجرد شركة ناشئة تكافح للحصول على أول ١٠ عملاء دوليين.

من هو البطل الحقيقي؟ الأقسام المتنافسة داخل الشركة
نظر كريم إلى الجدول الذي يقارن بين أداء الأقسام المختلفة. قسم البرمجيات كان يقود القافلة، محققًا ٤٥٠ مليون دولار، أي ما يعادل ٥٠٪ من إجمالي الإيرادات، بينما حقق قسم الأجهزة الذكية ٣٢٠ مليون دولار، والخدمات السحابية ١٢٥ مليون دولار. ولكن ما لفت انتباهه أكثر هو معدل النمو في الخدمات السحابية، الذي بلغ ٣٥٪، وهو ما جعله يدرك أن مستقبل الشركة قد يكون هناك، وليس في الأجهزة الذكية التي بدأت مبيعاتها في الانخفاض.

تحديات تهدد النجاح
لكن لم يكن كل شيء ورديًا. وجد كريم تقريرًا منفصلًا يتحدث عن المشكلات التي واجهتها الشركة. كان أولها انخفاض الطلب على الأجهزة الذكية بنسبة ٥٪، بسبب المنافسة الشديدة من شركات كبرى تقدم أجهزة أرخص بمواصفات مشابهة. كما أن تكاليف التشغيل ارتفعت بنسبة ١٠٪ بسبب التضخم وزيادة أجور الموظفين.

لكن المشكلة الأكبر التي أصابت الشركة هذا العام كانت تأخير إطلاق منتجها الجديد "ألترا-إكس". كان هذا الجهاز يُفترض أن يكون نقلة نوعية في عالم التكنولوجيا، لكن مشاكل تقنية أجبرت الفريق على تأجيله، مما أدى إلى انخفاض ثقة المستثمرين بنسبة ٧٪.

وكأن هذا لم يكن كافيًا، تعرض أحد تطبيقات الشركة لاختراق أمني أدى إلى تسريب بيانات ١٠٠ ألف مستخدم. لحل الأزمة، دفعت الشركة ١٥ مليون دولار كتعويضات للمتضررين، وهو مبلغ كان من الممكن أن يُستثمر في أشياء أكثر فائدة.

رؤية نحو المستقبل: هل يكون 2025 عامًا ذهبيًا؟
رغم كل هذه التحديات، لم يكن كريم من النوع الذي يستسلم بسهولة. مع انتهاء الاجتماع، بدأ في وضع خطة استراتيجية لعام ٢٠٢٥. كان أهم ما في الخطة إطلاق خدمة جديدة تعتمد على الذكاء الاصطناعي لتحليل البيانات، وهو مجال نامٍ ترى الشركة فيه فرصة كبيرة.

كما قررت "الرواد" التوسع في دول جديدة ( امريكا و المجر و صربيا و جنوب افريقيا و السنغال) في الشرق الأوسط وآسيا، والاستثمار ٥٠ مليون دولار في تعزيز الأمن السيبراني لتجنب أزمات مشابهة لما حدث في هذا العام. وأخيرًا، وافق مجلس الإدارة على توظيف ٣٠٠ موظف جديد لتعزيز فرق البحث والتطوير، لأن الابتكار كان ولا يزال مفتاح نجاح "الرواد".

وهو ينظر إلى مستقبل الشركة، تساءل كريم: هل سيكون العام القادم عامًا ذهبيًا لـ "الرواد"، أم أن التحديات ستزداد صعوبة؟
""".strip()

pprint(main_content)

('قصة نجاح وتحديات شركة "الرواد للتقنية" في عام 2024\n'
 'في صباح أحد الأيام الباردة من شهر يناير 2024، جلس كريم العلوي، المدير '
 'التنفيذي لشركة "الرواد للتقنية"، في مكتبه الزجاجي المطل على المدينة الصاخبة. '
 'كان العام الماضي عامًا مليئًا بالتحديات والإنجازات، وأمامه الآن تقرير الأداء '
 'السنوي، الذي سيحدد مستقبل الشركة في الأشهر القادمة.\n'
 '\n'
 'فتح كريم الملف، وابتسم عندما رأى الأرقام أمامه: ٨٩٥ مليون دولار هي الإيرادات '
 'السنوية للشركة، وهو رقم قياسي مقارنة بالأعوام السابقة. لكن ما أسعده أكثر هو '
 'تحقيق ١٢٥ مليون دولار كأرباح صافية، وهو ارتفاع بنسبة ١٥٪ عن العام الماضي. '
 'ورغم ذلك، لم تكن كل الأرقام بنفس الإيجابية، فقد زادت المصاريف التشغيلية إلى '
 '٤٢٠ مليون دولار، وهو ما جعل الشركة تبحث عن طرق لخفض التكاليف دون التأثير على '
 'جودة الخدمات.\n'
 '\n'
 'أحد أكبر استثمارات الشركة هذا العام كان في البحث والتطوير، حيث تم تخصيص ٣٥ '
 'مليون دولار لهذا القسم، على أمل أن تقود هذه الاستثمارات إلى ابتكارات جديدة '
 'تبقي "الرواد للتقنية" في مقدمة السوق.\n'
 '\n'
 'المنتجات وا

In [27]:
# ✅
question = 'كم تبلغ المصاريف التشغيلية للشركة؟'
gen_response_basic_template(main_content, question)

'The operational expenses for the company are $420 million.'

In [28]:
# ❌ The model sometimes answers in English.
question = 'كم مقدار التكاليف التي انفقتها الشركة لعمليات الابحاث؟'
gen_response_basic_template(main_content, question)

'The company spent 35 million dollars on research and development.'

In [ ]:
# ✅
question = 'اذا اعتمدنا على ارقام الايرادات. من سيكون القسم الأقل كفاءة؟'
gen_response_basic_template(main_content, question)

'بناءً على أرقام الإيرادات، يبدو أن قسم الأجهزة الذكية هو الأقل كفاءة، حيث حقق 320 مليون دولار، مقارنة بـ450 مليون دولار لقسم البرمجيات، الذي مثل 50% من إجمالي الإيرادات.'

In [29]:
# ✅
question = 'ما هو عدد الدول  الجديدة التي تعمل فيها الشركة حاليًا؟'
gen_response_basic_template(main_content, question)

'الشركة تعمل حاليًا في 45 دولة، وقد توسعت في دول جديدة (أمريكا، المجر، صربيا، جنوب أفريقيا، والسنغال) في الشرق الأوسط وآسيا.'

In [30]:
# ❌ The model is affected by the fake news in the question.
question = 'لماذ قررت الشركة فصل 50 موظف من فرق البحث و التطوير؟'
pprint(gen_response_basic_template(main_content, question))

('شركة "الرواد للتقنية" قررت فصل 50 موظف من فرق البحث والتطوير بسبب ارتفاع '
 'المصاريف التشغيلية إلى ٤٢٠ مليون دولار، مما جعل الشركة تبحث عن طرق لخفض '
 'التكاليف دون التأثير على جودة الخدمات.')
